<h1>ATMOS Version 1.1.0</h1>
The ATMOS calculator, written in Python. It is able to: 
<li>Determine atmos properties for any given altitude</li>
<li>Determine all airspeeds (including Mach) given a speed and altitude</li>
<li>Determine all airspeeds given Mach and altitude</li>



In [49]:
# Python Imports
import pandas as pd
import numpy as np
import math
import csv

<h2>Load the Standard Atmosphere</h2>
Imports standard atmosphere from a file "/instance/standard_atmos.csv".

Parameters are:
<li>Hg:       Geometric Altitude [ft]</li>
<li>H:        Geopotential Altitude [ft]</li>
<li>T:        Temperature [deg_R]</li>
<li>P:        Presure [lb/ft3]</li>
<li>P/P0:     Presure Ratio </li>
<li>pho:      Density [slug/ft3] </li>
<li>pho/pho0: Density Ratio </li>

In [59]:
# Open csv
df_atmos = pd.read_csv('./instance/standard_atmos.csv', comment="#")

print(df_atmos.head(3))


   Hgeo_ft    H_ft   T_degR  P_lb/ft3    P/P0  pho_slug/ft3  pho/pho0
0    -2500 -2500.0  527.584    2300.5  1.0871      0.002547    1.0711
1        0     0.0  518.670    2116.2  1.0000      0.002378    1.0000
2     2500  2500.0  509.756    1931.9  0.9129      0.002209    0.9289


<h2>User Defines the condition to analize</h2>
User selects
<li>Altitude pressure in ft</li>
<li>Speed KEAS, KCAS or KTAS</li>
<li>Speed value in Kts</li>

In [51]:
h_press_user = 60000 # Units ft
analysis_type = "Mach" # "KEAS", "KCAS", "KTAS" or Mach
user_speed = 0.8   # Units kts

<h2>Calculation of Atmosheric Parameters</h2>
Process flow:
<li>Extract sealevel contants</li>
<li>Test request is "on table"</li>
<li>Interpolate atmos properties to requested altitude</li>
<li>Caculate airspeeds and Mach</li>

In [52]:
# Extract P at sealevel
p_0 = df_atmos.loc[df_atmos["Hgeo_ft"]==0, "P_lb/ft3"].values[0] 
print(p_0)

2116.2


In [62]:
# Interpolate atmos properties to requested altitude
def standard_air_data (h_press, df_atmos):

  # Test request is "on table"
  hgeo_min = df_atmos["Hgeo_ft"].min()
  hgeo_max = df_atmos["Hgeo_ft"].max()

  # Test h within table
  if hgeo_min > h_press_user:
    print("ERROR: Altitude too low out of range")
    exit()
  elif hgeo_max < h_press_user:
    print("ERROR: Altitude too high out of range")
    exit()

  p_static  = np.interp(h_press, df_atmos["Hgeo_ft"], df_atmos["P_lb/ft3"])
  pho_ratio = np.interp(h_press, df_atmos["Hgeo_ft"], df_atmos["pho/pho0"])
  pho = np.interp(h_press, df_atmos["Hgeo_ft"], df_atmos["pho_slug/ft3"])

  print(f"For a requested altitude of: {h_press} ft")
  print(f"Pressure [lb/ft2]: {p_static}")
  print(f"Density [slug/ft3]: {pho}")
  
  df = {
    'h_press_ft':   h_press,
    'p_static_psf': p_static,
    'pho_ratio':    pho_ratio,
    'pho_slug_ft3': pho,
  }

  return df

user_atmos = standard_air_data(h_press=h_press_user, df_atmos=df_atmos)


For a requested altitude of: 60000 ft
Pressure [lb/ft2]: 151.03
Density [slug/ft3]: 0.000225691
0.0949


In [93]:
# Calculate Airspeed

def speed_calc (analysis, speed_defined, atmos, p_0):

  a_0        = 661.47  # kts
  gamma      = 1.4     # ratio of specific heats for air

  aTarget    = math.sqrt(gamma*atmos['p_static_psf']/atmos['pho_slug_ft3'])*0.5924838  # kts, p [], pho []

  if analysis.upper() == "KEAS":  # Method: ALT AND KEAS
    KEASTarget = speed_defined
    KTASTarget = KEASTarget/math.sqrt(atmos['pho_ratio'])
    MachTarget = KTASTarget/aTarget
    qTarget    = atmos['p_static_psf']*((1+0.2*MachTarget**2)**(7/2)-1)
    KCASTarget = a_0*math.sqrt(5*((qTarget/p_0+1)**(2/7)-1))

  elif analysis.upper() == "KTAS":  # Method ALT and KTAS
    KTASTarget = speed_defined
    KEASTarget = KTASTarget*math.sqrt(atmos['pho_ratio'])
    MachTarget = KTASTarget/aTarget
    qTarget    = atmos['p_static_psf']*((1+0.2*MachTarget**2)**(7/2)-1)
    KCASTarget = a_0*math.sqrt(5*((qTarget/p_0+1)**(2/7)-1))

  elif analysis.upper() == "KCAS":  # Method ALT and KCAS    
    KCASTarget = speed_defined
    qTarget =  p_0*((0.2*(KCASTarget/a_0)**2+1)**(7/2)-1)
    MachTarget = math.sqrt(5*((qTarget/atmos['p_static_psf']+1)**(2/7)-1))
    KTASTarget = aTarget*MachTarget
    KEASTarget = KTASTarget*math.sqrt(atmos['pho_ratio'])
  
  elif analysis.upper() == "MACH": #Method ALT and MACH
    MachTarget = speed_defined
    KTASTarget = aTarget*MachTarget
    KEASTarget = KTASTarget*math.sqrt(atmos['pho_ratio'])
    qTarget    = atmos['p_static_psf']*((1+0.2*MachTarget**2)**(7/2)-1)
    KCASTarget = a_0*math.sqrt(5*((qTarget/p_0+1)**(2/7)-1))

  else:
    print("Requested analysis not available")
    KTASTarget = np.NaN
    KCASTarget = np.NaN
    KEASTarget = np.NaN
    MachTarget = np.NaN
    qTarget = np.NaN

  print(f"Mach: {MachTarget}")

  if MachTarget > 1.0 and MachTarget != np.NaN:
    print("Mach greater than 1.0 quations not valid.")
    KTASTarget = np.NaN
    KCASTarget = np.NaN
    KEASTarget = np.NaN
    MachTarget = np.NaN
    qTarget = np.NaN

  df = {
    'Alt_ft': atmos['h_press_ft'],
    'KCAS': KCASTarget,
    'KEAS': KEASTarget,
    'KTAS': KTASTarget,
    'Mach': MachTarget,
    'q_psf': qTarget,
  }

  return df

output = speed_calc(
  analysis=analysis_type,
  speed_defined=user_speed,
  atmos=user_atmos,
  p_0=p_0,
  )

print(output)

Mach: 0.8
{'Alt_ft': 60000, 'KCAS': 151.9359453886258, 'KEAS': 141.33121156258534, 'KTAS': 458.78052680793803, 'Mach': 0.8, 'q_psf': 79.19107164364269}


<h2>Output</h2>
Prints output to user and appends to log file.

In [69]:


with open('AtmosLog.txt', 'w') as f:  
    w = csv.DictWriter(f, output.keys())
    w.writeheader()
    w.writerow(output)


<h2>Make a Speed Altitude Chart</h2>
We want the ability to convert between ALt and a speed (KEAS, KCAS, KTAS or Mach) and display lines of constant speed.

Initial capability is subsonic.

User defines the contant Speed points.

In [95]:
# Contant Speed defined
alt_range = [0.01, 5000, 10000, 15000, 20000, 30000, 40000, 50000]
const_keas = [0.01, 100, 200, 300, 400, 500, 600, 700]
const_kcas = [0.01, 100, 200, 300, 400, 500, 600, 700]
const_ktas = [0.01, 100, 200, 300, 400, 500, 600, 700]
const_mach = [0.01, 0.3, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95, 1.0]

# Initalize the output csv file
with open('speed_alt_data.csv', 'w') as f:  
    w = csv.DictWriter(f, output.keys())
    w.writeheader()

# Loop through the constant to get tables of constant data.

for alt in alt_range:

    atmos_data = standard_air_data (h_press=alt, df_atmos=df_atmos)

    # Calc all keas
    for eas in const_keas:
        
        output = speed_calc(
            analysis="keas",
            speed_defined=eas,
            atmos=atmos_data,
            p_0=p_0,
            )
        
        with open('speed_alt_data.csv', 'a') as f:  
            w = csv.DictWriter(f, output.keys())
            w.writerow(output)

    # Calc all ktas
    for tas in const_ktas:
        
        output = speed_calc(
            analysis="ktas",
            speed_defined=tas,
            atmos=atmos_data,
            p_0=p_0,
            )
        
        with open('speed_alt_data.csv', 'a') as f:  
            w = csv.DictWriter(f, output.keys())
            w.writerow(output)

    # Calc all kcas
    for cas in const_kcas:
        
        output = speed_calc(
            analysis="kcas",
            speed_defined=cas,
            atmos=atmos_data,
            p_0=p_0,
            )
        
        with open('speed_alt_data.csv', 'a') as f:  
            w = csv.DictWriter(f, output.keys())
            w.writerow(output)

    # Calc all Mach
    for mach in const_mach:
        
        output = speed_calc(
            analysis="mach",
            speed_defined=mach,
            atmos=atmos_data,
            p_0=p_0,
            )
        
        with open('speed_alt_data.csv', 'a') as f:  
            w = csv.DictWriter(f, output.keys())
            w.writerow(output)

For a requested altitude of: 0.01 ft
Pressure [lb/ft2]: 2116.1992628
Density [slug/ft3]: 0.002377651323548
Mach: 1.5120124521016549e-05
Mach: 0.15120124521016548
Mach: 0.30240249042033096
Mach: 0.45360373563049644
Mach: 0.6048049808406619
Mach: 0.7560062260508273
Mach: 0.9072074712609929
Mach: 1.0584087164711582
Mach greater than 1.0 quations not valid.
Mach: 1.5120122370934687e-05
Mach: 0.15120122370934688
Mach: 0.30240244741869376
Mach: 0.4536036711280406
Mach: 0.6048048948373875
Mach: 0.7560061185467344
Mach: 0.9072073422560812
Mach: 1.058408565965428
Mach greater than 1.0 quations not valid.
Mach: 1.5117850430081324e-05
Mach: 0.15117846209075758
Mach: 0.3023569233089351
Mach: 0.4535353828954862
Mach: 0.6047138402909747
Mach: 0.7558922951770068
Mach: 0.9070707474711432
Mach: 1.0582491972863746
Mach greater than 1.0 quations not valid.
Mach: 0.01
Mach: 0.3
Mach: 0.5
Mach: 0.6
Mach: 0.7
Mach: 0.8
Mach: 0.85
Mach: 0.9
Mach: 0.95
Mach: 1.0
For a requested altitude of: 5000 ft
Pressure [